In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_test = pd.read_csv('test.csv')

In [4]:
df_train.shape

(1460, 81)

In [5]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
numerical = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 
             'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
             'TotalBsmtSF', '1stFlrSF', '2ndFlrSF','LowQualFinSF', 
             'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
             'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'KitchenAbvGr',
             'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 
             'BedroomAbvGr', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
             'ScreenPorch', 'PoolArea', 'MiscVal']
ordinal = ['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
           'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
           'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 
           'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 
           'PavedDrive', 'PoolQC', 'Fence']
binary = ['CentralAir']
cyclical = ['MoSold', 'YrSold']
ids = ['Id'] 
categorical = set(df_train.columns).difference(set(numerical),
                                              set(ordinal), set(binary),
                                              set(cyclical), set(ids))


In [123]:
y_train = df_train.SalePrice
df_full = pd.concat([df_train.drop('SalePrice',axis=1), df_test])

In [124]:
################# DEALING WITH NUMERICAL VARIABLES ###########################

In [125]:
# Transform Month and Year sold into a single
df_full.MoSold = (df_full.MoSold - 1)/12
df_full['TimeSold'] = df_full.MoSold + df_full.YrSold
numerical.append('TimeSold')
df_full.drop(cyclical, axis=1, inplace=True)

In [83]:
df_train[numerical].head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageArea,WoodDeckSF,BedroomAbvGr,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,TimeSold
0,65.0,8450,2003,2003,196.0,706,0,150,856,856,...,548,0,3,61,0,0,0,0,0,2008.083333
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1262,...,460,298,3,0,0,0,0,0,0,2007.333333
2,68.0,11250,2001,2002,162.0,486,0,434,920,920,...,608,0,3,42,0,0,0,0,0,2008.666667
3,60.0,9550,1915,1970,0.0,216,0,540,756,961,...,642,0,3,35,272,0,0,0,0,2006.083333
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1145,...,836,192,4,84,0,0,0,0,0,2008.916667


In [87]:
df_train[numerical].isna().sum()[df_train[numerical].isna().any()]

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [94]:
df_train[numerical] =  df_train[numerical].fillna(0)
# These NAs are related to missing pieces in the house (no garage, no front)
# etc. 0 is adequate as it can be regressed on for areas, while it allows an
# easy split and separation between HAS - HAS NOT

In [99]:
df_full

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TimeSold
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,0.083333,2008,WD,Normal,2008.083333
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,0.333333,2007,WD,Normal,2007.333333
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,0.666667,2008,WD,Normal,2008.666667
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,0.083333,2006,WD,Abnorml,2006.083333
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,0.916667,2008,WD,Normal,2008.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6.000000,2006,WD,Normal,NaN
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4.000000,2006,WD,Abnorml,NaN
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9.000000,2006,WD,Abnorml,NaN
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7.000000,2006,WD,Normal,NaN
